# Data engenearing workbook

## Context:
My holdstaker needs to menage data from he's clients. 
I recived a pendrive with two main pathes, containg data from abou 120 clients
Total archive first count:

In path 'NF' all files has the same prefix 'NF'
In path 'PlanilhaCorretores' some files prefix ['Prestação de Contas', 'Planilha Comissão']

## Taks:
### - Merge 'pdf' files from same client
      - Identify and Extract client name in all prefix
      - Join match files as Client Name.pdf

### - Create a table containing information about clients: name, processNumber, situation, responsableOne, nf(bool)
      - match = extract and compare name from prefix
      - name = originalName, processNumber = (get originalName and processNumber from table in pdf file)
      - situation,responsableOne = 'Planilha Comissão 'Name' (situation, responsableOne)
      - nf (for client in clients, if nf 'name' in nf[], return true, else, false)


### - Create a table containing information about cliente in NF: name, nf_number, date  

In [125]:
import os
import re
import pandas as pd

PATH_NF = r'D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Toledo 2025_Edit\N Fiscais Toledo 2025'
PATH_PLANILHAS_PRESTACOES = r'D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Toledo 2025_Edit\Planilha Corretores 2025'


nf_table = {
    "origem": [],
    "nome": [],
    "data": [],
    "corretor": [],
    "situação": []
}

pr_table = {
    "origem": [],
    "nome": [],
    "n_processo":[],
}

pl_table = {
    "origem": [],
    "nome": [],
    "corretor": [],
    "situação": []
}

table = {
    "origem": [],
    "nome": [],
    "n_processo":[],
    "situação": [],
    "corretor": [],
    "nf_exist": [],       
}

for fullPath in [PATH_PLANILHAS_PRESTACOES, PATH_NF]:
    for file in os.listdir(fullPath):
        if file.lower().endswith('.pdf'):
            table["origem"].append(fullPath + '\\' + file)
            table["nome"].append(file)
        if file.lower().startswith('nf '):
            nf_table["origem"].append(fullPath + '\\' + file)
            nf_table["nome"].append(file)
        if file.lower().startswith('planilha'):
            pl_table["origem"].append(fullPath + '\\' + file)
            pl_table["nome"].append(file)
        if file.lower().startswith('prestação'):
            pr_table["origem"].append(fullPath + '\\' + file)
            pr_table["nome"].append(file)

In [126]:
print('Resumo da Análise de Arquivos PDF:')
print(f'Total de arquivos: {len(table["origem"])}')
print(f'Total de NF: {len(nf_table["origem"])}')
print(f'Total de Planilhas: {len(pl_table["origem"])}')
print(f'Total de Prestações: {len(pr_table["origem"])}')

Resumo da Análise de Arquivos PDF:
Total de arquivos: 403
Total de NF: 145
Total de Planilhas: 122
Total de Prestações: 136


In [127]:
def processar_nome(nome):
    nome = nome.title()

    # Prefixos para remover
    for prefix in ['Planilha Comissão ', 'Prestação De Contas ', 'Nf ']:
        if nome.startswith(prefix.title()):
            nome = nome.replace(prefix.title(), '')

    # Remove extensão e parênteses
    nome = nome.replace('.Pdf', '')
    nome = re.sub(r'\s*\(.*?\)', '', nome).strip()

    # Tratamento de Espólio
    if nome.lower().startswith(("espólio", "espolio")):
        nome = re.sub(r'^(Esp[óo]lio( De)?)\s*', '', nome, flags=re.IGNORECASE)
        nome = nome.strip() + " - Espólio"

    return nome

In [128]:
# Extract date and nf_number from NF files
import PyPDF2

reader = PyPDF2.PdfReader(r"D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Toledo 2025_Edit\N Fiscais Toledo 2025\NF Abrahão Sadigursky.pdf")
texto = ""
for page in reader.pages:
    texto += page.extract_text()
if texto:
    print(texto)
else:
    print("No text found in the PDF.")

# No return in text extraction
# Suspecting that the PDF might be scanned images
# Need to use OCR to extract text from images in PDF

In [129]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import sys

# --- CONFIGURAÇÃO ---
# Caminho do executável do Tesseract
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH


def extrair_texto_pdf_ocr(file):
    texto_completo = ""
    try:
        # Abre o PDF
        doc = fitz.open(file)
        

        for i, pagina in enumerate(doc):
            # 1. Zoom para melhorar a qualidade da imagem para o OCR
            zoom = 2 
            matriz = fitz.Matrix(zoom, zoom)
            pix = pagina.get_pixmap(matrix=matriz)
            
            # 2. Converte para imagem PIL
            imagem = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # 3. Tenta fazer o OCR
            try:
                # Tenta ler em PORTUGUÊS primeiro
                texto_pagina = pytesseract.image_to_string(imagem, lang='eng')
            except pytesseract.TesseractError as e:
                # Se der erro de "language not found", cai aqui
                print(f"\nAviso: {e}. Tentando OCR em INGLÊS.")

            texto_completo += texto_pagina
            

        doc.close()

    except Exception as e:
        print(f"\nERRO CRÍTICO: {e}")
        print("\nVerifique se o caminho na linha 9 está apontando para onde você instalou o Tesseract.")

    return texto_completo


def extrair_data_texto(texto):
    padrao_data = r"Data e Hora de Emiss[ãa]o.*?:?\s*\"?(\d{2}/\d{2}/\d{4})"
    match = re.search(padrao_data, texto, re.IGNORECASE | re.DOTALL)

    if match:
        return match.group(1)        
    else:
        print("N/A.")
        return None
        

    return data_encontrada

In [130]:
for o in nf_table['origem']:
    texto = extrair_texto_pdf_ocr(o)
    data = extrair_data_texto(texto)    
    nf_table["data"].append(data)

N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.
N/A.


In [131]:
nf_table['nome'] = [processar_nome(n) for n in nf_table['nome']]
for k, v in nf_table.items():
    print(f"{k}: {v}")

origem: ['D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Abrahão Sadigursky.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Adalgiso Silva Carvalho.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Aida Pedreira Sampaio.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alain Peixoto de Jesus.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alberto F. Tamandaré.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alceu Vila Nova Maciel Filho (CANCELADA POR DUPLICIDADE).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alceu Vila Nova Maciel Filho (Vl Total).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025

In [138]:
def extrair_situacao(texto):
    padrao = r'\(\s*([^\s)]+)\s+([^)]+)\)'
    match = re.search(padrao, texto)

    if match:
        return match.group(1)
    else:
        print ("Verificar situação.")
        return None            

def extrair_corretor(texto):
    padrao = r'\(\s*([^\s)]+)\s+([^)]+)\)'
    match = re.search(padrao, texto)

    if match:
        return match.group(2).strip()
    else:
        print ("Verificar corretor.")
        return None

In [133]:
pl_table['nome'] = [processar_nome(n) for n in pl_table['nome']]
pl_table['corretor'] = [extrair_corretor(t) for t in pl_table['origem']]
pl_table['situação'] = [extrair_situacao(t) for t in pl_table['origem']]
for k, v in pl_table.items():
    print(f"{k}: {v}")

Verificar corretor.
Verificar corretor.
Verificar situação.
Verificar situação.
origem: ['D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\Planilha Corretores 2025\\Planilha Comissão  ESPOLIO DE ANANIAS NOVAES DE SOUZA rep Lilian Calmon Novais (VT Lilliam).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\Planilha Corretores 2025\\Planilha Comissão Abrahão Sadigursky (Ac Beto).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\Planilha Corretores 2025\\Planilha Comissão Adalgiso Silva Carvalho (VT Edson e Noé).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\Planilha Corretores 2025\\Planilha Comissão Alain Peixoto de Jesus (VT Lilliam).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\Planilha Corretores 2025\\Planilha Comissão Alberto F. Tamandaré (VF Roberto).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\Planilha Corretores 2

In [134]:
for nf in nf_table['nome']:
    encontrado = False
    for i, pl in enumerate(pl_table['nome']):
        if nf == pl:
            nf_table['corretor'].append(pl_table['corretor'][i])
            nf_table['situação'].append(pl_table['situação'][i])
            encontrado = True            
            break
    if not encontrado:
        nf_table['corretor'].append(None)

for k, v in nf_table.items():
    print(f"{k}: {v}")

origem: ['D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Abrahão Sadigursky.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Adalgiso Silva Carvalho.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Aida Pedreira Sampaio.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alain Peixoto de Jesus.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alberto F. Tamandaré.pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alceu Vila Nova Maciel Filho (CANCELADA POR DUPLICIDADE).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025_Edit\\N Fiscais Toledo 2025\\NF Alceu Vila Nova Maciel Filho (Vl Total).pdf', 'D:\\Documentos\\JOBS\\DATA\\RelatórioAndiappToledo\\Toledo 2025

In [135]:
for k, v in nf_table.items():
    print(k, len(v))

origem 145
nome 145
data 145
corretor 145
situação 125


In [139]:
import pandas as pd

# --- CONFIGURE A PASTA AQUI ---
OUTPUT_FOLDER = r"D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Dados\Relação_NF"

# garante que a pasta existe
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# cria caminho completo do arquivo
output_path = os.path.join(OUTPUT_FOLDER, "nf_table.csv")
max_len = max(len(v) for v in nf_table.values())

# completa automaticamente
for k, v in nf_table.items():
    if len(v) < max_len:
        v.extend([None] * (max_len - len(v)))
# exporta
df = pd.DataFrame(nf_table)
df['situação'] = df['situação'].fillna('N/A')
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"CSV criado em: {output_path}")

CSV criado em: D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Dados\Relação_NF\nf_table.csv


In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import sys

# --- CONFIGURAÇÃO ---
# Caminho do executável do Tesseract
caminho_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = caminho_tesseract

# Caminho do seu PDF
arquivo_pdf = r"D:\Documentos\JOBS\DATA\RelatórioAndiappToledo\Toledo 2025_Edit\N Fiscais Toledo 2025\NF Abrahão Sadigursky.pdf"

texto_completo = ""

try:
    # Abre o PDF
    doc = fitz.open(arquivo_pdf)
    print(f"Lendo {len(doc)} páginas do arquivo...")

    for i, pagina in enumerate(doc):
        # 1. Zoom para melhorar a qualidade da imagem para o OCR
        zoom = 2 
        matriz = fitz.Matrix(zoom, zoom)
        pix = pagina.get_pixmap(matrix=matriz)
        
        # 2. Converte para imagem PIL
        imagem = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # 3. Tenta fazer o OCR
        try:
            # Tenta ler em PORTUGUÊS primeiro
            texto_pagina = pytesseract.image_to_string(imagem, lang='por')
        except pytesseract.TesseractError as e:
            # Se der erro de "language not found", cai aqui
            if "Failed loading language" in str(e) or "Error opening data file" in str(e):
                print(f"\n[AVISO] Pacote de idioma 'Português' não encontrado no Tesseract.")
                print("[AVISO] Mudando para 'Inglês' (padrão). Acentos podem sair errados.\n")
                # Tenta ler em INGLÊS (fallback)
                texto_pagina = pytesseract.image_to_string(imagem, lang='eng')
            else:
                raise e # Se for outro erro, para tudo

        texto_completo += texto_pagina
        print(f"Página {i+1} processada.")

    doc.close()

    print("\n" + "="*40)
    print("CONTEÚDO EXTRAÍDO")
    print("="*40)
    print(texto_completo)

except Exception as e:
    print(f"\nERRO CRÍTICO: {e}")
    print("\nVerifique se o caminho na linha 9 está apontando para onde você instalou o Tesseract.")

Lendo 1 páginas do arquivo...

ERRO CRÍTICO: [WinError 32] O arquivo já está sendo usado por outro processo: 'C:\\Users\\pedro\\AppData\\Local\\Temp\\tess_q74pw5mj_input.PNG'

Verifique se o caminho na linha 9 está apontando para onde você instalou o Tesseract.


SUCESSO! Data extraída: 08/03/2023


In [ ]:
# Split Client (PR, PL, NF)
lista_pr = []
lista_pl = []
lista_nf = []
for name, file in client_names:
    prefixo = os.path.basename(file)[0:2].upper()
    if prefixo == 'PR':
        lista_pr.append((name, file))
    elif prefixo == 'PL':
        lista_pl.append((name, file))
    elif prefixo == 'NF':
        lista_nf.append((name, file))

# Criar conjunto com todos os clientes
todos_clientes = sorted(set([c for c, _ in lista_pr + lista_pl + lista_nf]))

# Gerar lista_ok
lista_ok = []
lista_check = []
for cliente in todos_clientes:
    origens = []
    if cliente in [c for c, _ in lista_pr]:
        origens.append('PR')
    if cliente in [c for c, _ in lista_pl]:
        origens.append('PL')
    if cliente in [c for c, _ in lista_nf]:
        origens.append('NF')

    if len(origens) >= 2:
        lista_ok.append(cliente)
    else:
        lista_check.append(cliente)



print(len(lista_check))
for c in sorted(lista_check):
    print(c)

NameError: name 'client_names' is not defined

In [ ]:
for c in nf:
    print(c)

NameError: name 'nf' is not defined